# [CS303]Practice 8 - Local Search: Genetic Algorithm
In this practice, you will explore how **Genetic Algorithms (GA)** work and how they can be used to search for good solutions in large, complex spaces.

You will first experiment with a **toy problem** — generating a target phrase — and then apply the same idea to solve the **N-Queens problem**.

📌 **DDL: Nov.23th**

## Evaluation Criteria

Your submission will be assessed on:

* Clear understanding of every line of your code (no blind copy-paste).
* Successful compilation of the program.
* Correctness of the program logic.
* Reasonable efficiency of the solution.

### Grading Policy

* **On-time submission:** 60–100 points
* **Late submission:** 0 points




In [1]:
%load_ext autoreload
%autoreload 2
import random
import numpy as np
import matplotlib.pyplot as plt
from utils import plot_NQueens, plot_evolution

## 1. Generating Phrases

Let’s begin with a simple but fun example: evolving random strings until one of them matches a **target phrase** (e.g., `"Genetic Algorithm"`).
Assume the length of the target phrase is known.

### Problem Formulation

1. **Representation:** Each individual is a string.
2. **Search Space:**  $m^l$ where $m$ is the size of the alphabet (set of characters in our interest) and $l$ is the length of the string.

### Designing a Genetic Algorithm

1. **Chromosome:** The string itself.
2. **Fitness Function:** Measures how close a string is to the target phrase (e.g., number of matching characters).
3. **Crossover (Mating):** Combines two parent strings to produce a new string.
4. **Mutation:** Randomly changes a few characters to maintain diversity in the population.

The following cells will guide you through the process step by step. Complete the `TODO` sections in the provided code so the program runs correctly — this will show how a GA gradually evolves a population toward the target phrase.


In [2]:
# setting up the problem
target = 'Genetic Algorithm'
u_case = [chr(x) for x in range(65, 91)]
l_case = [chr(x) for x in range(97, 123)]
gene_pool = u_case + l_case + [' '] # all English chracters and white space

def init_population(pop_size, gene_pool, state_length):
    """
    Randomly initialize a population for genetic algorithm
        pop_size  :  Number of individuals in population
        gene_pool   :  List of possible values for individuals
        state_length:  The length of each individual
    """

    population = []
    for _ in range(pop_size):
        new_individual = "".join(random.choices(gene_pool, k=state_length))
        population.append(new_individual)

    return population

# test
init_pop = init_population(10, gene_pool, len(target))
init_pop

['lPyUGNxeVFEoMvYXX',
 'zAOfycgAawkesHZmO',
 'ECoQC eLhpblneYWJ',
 'Ecxdsrfri sTmaUop',
 'tdVxURtoCqtzWFmyu',
 'eTF oCqRVpOipZTCg',
 'XNCiJcHvO nEnBjDh',
 'yUmyaJmXYZAIUTyum',
 'daXOFbrLqiACg pdH',
 'gLTxpYRfU MSHxpLB']

In [ ]:
def fitness_fn(sample):
    # TODO: evaluate how close *sample* is to the target, e.g., number of matching characters
    raise NotImplementedError()

# test
fitness_fn(init_pop[0])

In [ ]:
def select(r, population, fitness_fn):
    """
    TODO: select *r* samples from *population*
    the simplest choice is to sample from *population* with each individual weighted by its fitness
    """
    raise NotImplementedError()

# test
parents = select(2, init_pop, fitness_fn)
parents

In [ ]:
def recombine(x, y):
    """
    TODO: combine two parents to produce an offspring
    """
    raise NotImplementedError()

def mutate(x, gene_pool, pmut):
    """
    apply mutation to *x* by randomly replacing one of its gene from *gene_pool*
    """
    if random.uniform(0, 1) >= pmut:
        return x

    n = len(x)
    g = len(gene_pool)
    c = random.randrange(0, n)
    r = random.randrange(0, g)

    new_gene = gene_pool[r]
    return x[:c] + new_gene + x[c + 1:]

# test
child = mutate(recombine(*parents), gene_pool, 0.1)
child

In [ ]:
# now refactor things into a *Problem* abstraction
# you can directly reuse what you have implemented above
from abc import ABC, abstractmethod

class GAProblem(ABC):
    @abstractmethod
    def init_population(self, pop_size): pass

    @abstractmethod
    def fitness(self, sample): pass

    @abstractmethod
    def reproduce(self, population): pass

    @abstractmethod
    def replacement(self, old, new): pass

class PhraseGeneration(GAProblem):
    def __init__(self, target, alphabet):
        self.target = target
        self.alphabet = alphabet

    def init_population(self, pop_size):
        # raise NotImplementedError()
        return init_population(pop_size, self.alphabet, len(self.target))

    def fitness(self, sample):
        # TODO: evaluate how close *sample* is to the target
        raise NotImplementedError()
    
    def reproduce(self, population, mutation_rate):
        """
        TODO: generate the next generation of population

        hint: make a new individual with 
        
        mutate(recombine(*select(2, population, fitness_fn)), gene_pool, pmut)

        """
        raise NotImplementedError()
    
    def replacement(self, old, new):
        """
        You can use your own strategy, for example retain some solutions from the old population
        """
        return new

def genetic_algorithm(
        problem: GAProblem, 
        ngen, n_init_size, mutation_rate, 
        log_intervel=100
    ):

    population = problem.init_population(n_init_size)
    best = max(population, key=problem.fitness)
    history = [(0, list(map(problem.fitness, population)))]

    for gen in range(ngen):
        next_gen    = problem.reproduce(population, mutation_rate)
        population  = problem.replacement(population, next_gen)

        if gen % log_intervel == 0:
            current_best = max(population, key=problem.fitness)
            if problem.fitness(current_best) > problem.fitness(best): best = current_best
            print(f"Generation: {gen}/{ngen},\tBest: {best},\tFitness={problem.fitness(best)}")         
            history.append((gen, list(map(problem.fitness, population))))
    
    history.append((ngen-1, list(map(problem.fitness, population))))
    return best, history

In [ ]:
# now set up the parameters
ngen = 1200
max_population = 120
mutation_rate = 0.08

sid = 11810424 #TODO:  replace this with your own sid
target = f"Genetic Algorithm by {sid}" 
alphabet = None # TODO: fix this: what is the search space now?

problem = PhraseGeneration(target, alphabet)

# and run it
solution, history = genetic_algorithm(problem, ngen, max_population, mutation_rate)
solution

In [ ]:
# visualize the evolution of the polulation
bins = np.linspace(0, problem.max_fitness, problem.max_fitness+1)
plot_evolution(history, bins)
bins

## 2. N-Queens Problem

Next, we will apply the genetic algorithm to the classic **N-Queens problem**.

### Problem Formulation

1. **Representation:** The board configuration, typically represented as an array indicating queen positions.
2. **Search Space:** $n^n$, if no constraints are applied.

### Modifying the GA

To adapt GA for this problem:

* The **fitness function** should measure how many pairs of queens are not attacking each other — the goal is to **maxmize** this value.
* Other parts (representation, crossover, mutation) can remain mostly the same as in the previous example.

By comparing the two problems, you’ll see how GAs can be adapted to very different types of search spaces.


In [ ]:
class NQueensProblem(GAProblem):
    def __init__(self, n):
        self.n = n
        self.max_fitness =  n*(n-1)//2 # max number if non-attacking pairs

    def init_population(self, pop_size):
        # TODO:alomost the same as the previous problem.
        raise NotImplementedError()
    
    def fitness(self, queens):
        """
        TODO

        hint: count the non-attacking pairs
        """
        raise NotImplementedError()

    def reproduce(self, population, mutation_rate):
        # TODO:alomost the same as the previous problem.
        raise NotImplementedError()

    def replacement(self, old, new):
        """
        You can use your own strategy, for example retain some solutions from the old population
        """
        return new

    def __repr__(self):
        return f"{self.n}-Queens Problem"

In [ ]:
from utils import plot_NQueens
ngen = 1000
init_size = 120
mutation_rate = 0.08

n = 8
problem = NQueensProblem(n)
solution, history = genetic_algorithm(problem, ngen, init_size, mutation_rate)

In [ ]:
# Example of how to use this function
# plot_NQueens([4, 2, 0, 6, 1, 7, 5, 3])
# replace the parameter with your own results
plot_NQueens(solution)

In [ ]:
# Visualize the evolution of the polulation
bins = np.linspace(0, problem.max_fitness, problem.max_fitness)
plot_evolution(history, bins)